<a href="https://colab.research.google.com/github/ManojKumarKolli/ManojKumar_INFO5731_Spring2024/blob/main/In_class_exercise/Kolli_ManojKumar_Exercise_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 2**

The purpose of this exercise is to understand users' information needs, and then collect data from different sources for analysis by implementing web scraping using Python.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)
Describe an interesting research question (or practical question or something innovative) you have in mind, what kind of data should be collected to answer the question(s)? Specify the amount of data needed for analysis. Provide detailed steps for collecting and saving the data.

In [ ]:
# write your answer here
"""
Research Question: Explore how public sentiment towards renewable energy technologies (solar, wind, hydroelectric) varies by region and changes over time on social media.

Data Required:

Social Media Posts: Related to solar, wind, and hydroelectric energy.
Sentiment Data: Sentiment scores (positive, neutral, negative) for each post.
Geographic and Temporal Data: Location (if available) and timestamps of posts.
Amount of Data: Minimum 3,000 samples (1,000 for each energy type) for a comprehensive analysis.

Collection Steps:

API Access: Obtain access to a social media platform's API, such as Reddit.
Data Collection: Use the API to collect posts mentioning specific renewable energy keywords, capturing text, sentiment, location, and time.
Data Storage: Save the data in a structured format, like a CSV file, with columns for each required piece of information.
Ethical Considerations: Follow the platform's data use policy and consider user privacy.


"""

## Question 2 (10 Points)
Write Python code to collect a dataset of 1000 samples related to the question discussed in Question 1.

In [2]:
pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.6 MB/s eta 0:00:00


In [8]:
# write your answer here
import praw
import pandas as pd

reddit = praw.Reddit(client_id='Rm51RgkkORs6zhyg23rNcg',
                     client_secret='G4yNrSb24cbkLUXm5frLVOxjji18Bg',
                     user_agent='test')

def collect_samples(subreddit, keyword, limit=1000):

    posts = []
    for submission in reddit.subreddit(subreddit).search(keyword, limit=limit):
        posts.append({
            'title': submission.title,
            'text': submission.selftext,
            'created_utc': submission.created_utc,
            'upvotes': submission.score,
            'num_comments': submission.num_comments
        })

    df = pd.DataFrame(posts)
    return df

df_solar = collect_samples('renewableenergy', 'solar energy', 20)
print(df_solar)
df_solar.to_csv('solar_energy_posts.csv', index=False)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



                                                title  \
0   Why solar energy will continue to lead the pac...   
1         The momentum of the solar energy transition   
2   Solar Energy Sites Nurture Blooming Insect Pop...   
3   The Annular Solar Eclipse Will Decimate US Sol...   
4     Harvesting more solar energy with supercrystals   
5       Minnesota approves giant solar energy project   
6   Dillon solar energy farm ushers in clean energ...   
7   Solar energy in the U.S. may triple in five years   
8   Producing solar energy, strawberries, tomatoes...   
9   How Solar Energy Can Help Boost Biodiversity A...   
10  Local city government thwarting solar energy g...   
11  Solar energy production tripled in Latvia over...   
12  Solar energy companies say scammers are going ...   
13  Bureau of Land Management analysis aims to opt...   
14  Egypt constructs solar energy stations in muse...   
15  500-mile-long solar energy power line find fos...   
16  US could finance solar powe

## Question 3 (10 Points)
Write Python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "XYZ". The articles should be published in the last 10 years (2014-2024).

The following information from the article needs to be collected:

(1) Title of the article

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [7]:
# write your answer here
import requests
import xml.etree.ElementTree as ET
import time
import warnings
warnings.filterwarnings("ignore")


def fetch_arxiv_articles(keyword, max_results=1000):

    articles = []
    base_url = "http://export.arxiv.org/api/query?"
    start = 0
    max_per_query = 100

    while len(articles) < max_results:
        query_params = f"search_query=all:{keyword}&start={start}&max_results={max_per_query}"
        response = requests.get(base_url + query_params)

        if response.status_code == 200:
            root = ET.fromstring(response.content)
            for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
                title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
                published = entry.find('{http://www.w3.org/2005/Atom}published').text[:4]  # Extract year
                authors = [author.find('{http://www.w3.org/2005/Atom}name').text for author in
                           entry.findall('{http://www.w3.org/2005/Atom}author')]
                abstract = entry.find('{http://www.w3.org/2005/Atom}summary').text.strip()

                articles.append({
                    "title": title,
                    "venue/journal/conference": "arxiv",
                    "year": published,
                    "authors": ", ".join(authors),
                    "abstract": abstract
                })

                if len(articles) >= max_results:
                    break
            start += max_per_query
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

        time.sleep(3)

    return articles


keyword = "NLP"
articles = fetch_arxiv_articles(keyword, 10)
print("Retrieved Articles:")
for article in articles:
    print(article)
print()
print("*"*100)
print(f"Collected {len(articles)} articles")


Retrieved Articles:
{'title': 'Spark NLP: Natural Language Understanding at Scale', 'venue/journal/conference': 'arxiv', 'year': '2021', 'authors': 'Veysel Kocaman, David Talby', 'abstract': 'Spark NLP is a Natural Language Processing (NLP) library built on top of\nApache Spark ML. It provides simple, performant and accurate NLP annotations\nfor machine learning pipelines that can scale easily in a distributed\nenvironment. Spark NLP comes with 1100 pre trained pipelines and models in more\nthan 192 languages. It supports nearly all the NLP tasks and modules that can\nbe used seamlessly in a cluster. Downloaded more than 2.7 million times and\nexperiencing nine times growth since January 2020, Spark NLP is used by 54% of\nhealthcare organizations as the worlds most widely used NLP library in the\nenterprise.'}
{'title': 'Sejarah dan Perkembangan Teknik Natural Language Processing (NLP) Bahasa\n  Indonesia: Tinjauan tentang sejarah, perkembangan teknologi, dan aplikasi NLP\n  dalam baha

We have retrieved only 10 records instead of 10000 as we have limit constraints.

## Question 4A (10 Points)
Develop Python code to collect data from social media platforms like Reddit, Instagram, Twitter (formerly known as X), Facebook, or any other. Use hashtags, keywords, usernames, or user IDs to gather the data.



Ensure that the collected data has more than four columns.


In [5]:
import praw
import datetime

reddit = praw.Reddit(client_id='Rm51RgkkORs6zhyg23rNcg',
                     client_secret='G4yNrSb24cbkLUXm5frLVOxjji18Bg',
                     user_agent='test')


def fetch_reddit_posts(subreddit_name, keyword, limit=100):

    posts = []
    subreddit = reddit.subreddit(subreddit_name)

    # Search for submissions in the subreddit
    for submission in subreddit.search(keyword, limit=limit):
        post_details = {
            'title': submission.title,
            'url': submission.url,
            'created_utc': datetime.datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            'upvotes': submission.score,
            'num_comments': submission.num_comments,
            'author': str(submission.author)
        }
        posts.append(post_details)

    return posts


subreddit_name = 'NLP'
keyword = 'nlp'
posts = fetch_reddit_posts(subreddit_name, keyword, limit=10)

print("Retrieved posts:")
for post in posts:
    print(post)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Retrieved posts:
{'title': 'Why the "NLP" that you were taught is not real NLP if you did NOT train with a Licensed NLP trainer.', 'url': 'https://v.redd.it/g1mppkz1kmqb1', 'created_utc': '2023-09-26 16:14:43', 'upvotes': 1, 'num_comments': 20, 'author': 'JoostvanderLeij'}
{'title': 'NLP Power Words', 'url': 'https://www.reddit.com/r/NLP/comments/1ahk6lx/nlp_power_words/', 'created_utc': '2024-02-03 01:30:25', 'upvotes': 5, 'num_comments': 15, 'author': 'ihatewands'}
{'title': 'NLP For Porn Addiction & Success In Life', 'url': 'https://www.reddit.com/r/NLP/comments/1722kyz/nlp_for_porn_addiction_success_in_life/', 'created_utc': '2023-10-07 09:53:46', 'upvotes': 5, 'num_comments': 69, 'author': 'Illustrious_Car5155'}
{'title': 'NLP strategies for ADHD', 'url': 'https://www.reddit.com/r/NLP/comments/18mhsie/nlp_strategies_for_adhd/', 'created_utc': '2023-12-20 01:02:40', 'upvotes': 4, 'num_comments': 20, 'author': 'Capable-Breakfast480'}
{'title': 'NLP for comedy', 'url': 'https://www.r

## Question 4B (10 Points)
If you encounter challenges with Question-4 web scraping using Python, employ any online tools such as ParseHub or Octoparse for data extraction. Introduce the selected tool, outline the steps for web scraping, and showcase the final output in formats like CSV or Excel.



Upload a document (Word or PDF File) in any shared storage (preferably UNT OneDrive) and add the publicly accessible link in the below code cell.

Please only choose one option for question 4. If you do both options, we will grade only the first one

In [ ]:
# write your answer here


# Mandatory Question

**Important: Reflective Feedback on Web Scraping and Data Collection**



Please share your thoughts and feedback on the web scraping and data collection exercises you have completed in this assignment. Consider the following points in your response:



Learning Experience: Describe your overall learning experience in working on web scraping tasks. What were the key concepts or techniques you found most beneficial in understanding the process of extracting data from various online sources?



Challenges Encountered: Were there specific difficulties in collecting data from certain websites, and how did you overcome them? If you opted for the non-coding option, share your experience with the chosen tool.



Relevance to Your Field of Study: How might the ability to gather and analyze data from online sources enhance your work or research?

**(no grading of your submission if this question is left unanswered)**

In [ ]:
'''
Learning experience: The overall experience with the web scraping is excellent. We learnt about the html tags and scrapping based on the component. With the knowledge gained we can collect the dataset from various sources.

Challenges Encountered: We have some issue with the dynamic pages. So, we need to work more on the pagination in the query params.

Relevance to Your Field of Study: As a data science student, we can collect the data on our own and employ our ML, DL and NLP models.



'''